In [ ]:
import aiohttp
from aiohttp import web
import asyncio
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import os
import logging
import nest_asyncio
from aiohttp_swagger3 import SwaggerDocs, SwaggerUiSettings, SwaggerInfo

# Apply nest_asyncio to allow nested event loops in Jupyter Notebook
nest_asyncio.apply()

# Настройка логирования
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

# Словарь для хранения состояния выполнения для каждого этапа
progress = {
    'run_scrape': 0,
    'run_summarization': 0,
    'run_grade': 0,
    'run_digest_generation': 0
}

# Функция для выполнения Jupyter notebook
async def execute_notebook(notebook_paths, progress_key):
    progress[progress_key] = 0
    try:
        for notebook_path in notebook_paths:
            logger.info(f"Starting execution of notebook: {notebook_path}")
            with open(notebook_path) as f:
                nb = nbformat.read(f, as_version=4)
            ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
            await asyncio.get_event_loop().run_in_executor(None, ep.preprocess, nb, {'metadata': {'path': './'}})
            logger.info(f"Successfully executed notebook: {notebook_path}")
        progress[progress_key] = 100
    except Exception as e:
        progress[progress_key] = -1
        logger.error(f"Failed to execute notebook {notebook_path}: {e}", exc_info=True)

async def run_scrape(request):
    """
    ---
    description: Run the scrape notebooks
    tags:
    - Notebook Execution
    parameters:
    - name: token
      in: query
      required: true
      schema:
        type: string
    responses:
      '200':
        description: Scrape notebook execution started
      '400':
        description: Notebook not found
      '403':
        description: Invalid token
    """
    token = request.query.get('token')
    if token != 'your_secret_token':
        return web.json_response({'error': 'Invalid token'}, status=403)

    notebook_paths = ['rss.ipynb', 'scrape.ipynb', 'news_extraction.ipynb']
    for path in notebook_paths:
        if not os.path.exists(path):
            return web.json_response({'error': f'Notebook {path} not found'}, status=400)

    asyncio.create_task(execute_notebook(notebook_paths, 'run_scrape'))
    return web.json_response({
        'status': 'Scrape notebook execution started',
        'notebooks': notebook_paths,
        'message': 'Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867'
    }, status=200)

async def run_summarization(request):
    """
    ---
    description: Run the summarization notebook
    tags:
    - Notebook Execution
    parameters:
    - name: token
      in: query
      required: true
      schema:
        type: string
    responses:
      '200':
        description: Summarization notebook execution started
      '400':
        description: Notebook not found
      '403':
        description: Invalid token
    """
    token = request.query.get('token')
    if token != 'your_secret_token':
        return web.json_response({'error': 'Invalid token'}, status=403)

    notebook_path = 'news_range_n_summary.ipynb'
    if not os.path.exists(notebook_path):
        return web.json_response({'error': 'Notebook not found'}, status=400)

    asyncio.create_task(execute_notebook([notebook_path], 'run_summarization'))
    return web.json_response({
        'status': 'Summarization notebook execution started',
        'notebook': notebook_path,
        'message': 'Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867'
    }, status=200)

async def run_grade(request):
    """
    ---
    description: Run the grade notebook
    tags:
    - Notebook Execution
    parameters:
    - name: token
      in: query
      required: true
      schema:
        type: string
    responses:
      '200':
        description: Grade notebook execution started
      '400':
        description: Notebook not found
      '403':
        description: Invalid token
    """
    token = request.query.get('token')
    if token != 'your_secret_token':
        return web.json_response({'error': 'Invalid token'}, status=403)

    notebook_path = 'news_range_n_summary.ipynb'
    if not os.path.exists(notebook_path):
        return web.json_response({'error': 'Notebook not found'}, status=400)

    asyncio.create_task(execute_notebook([notebook_path], 'run_grade'))
    return web.json_response({
        'status': 'Grade notebook execution started',
        'notebook': notebook_path,
        'message': 'Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867'
    }, status=200)

async def run_digest_generation(request):
    """
    ---
    description: Run the digest generation notebook
    tags:
    - Notebook Execution
    parameters:
    - name: token
      in: query
      required: true
      schema:
        type: string
    responses:
      '200':
        description: Digest generation notebook execution started
      '400':
        description: Notebook not found
      '403':
        description: Invalid token
    """
    token = request.query.get('token')
    if token != 'your_secret_token':
        return web.json_response({'error': 'Invalid token'}, status=403)

    notebook_path = 'digest.ipynb'
    if not os.path.exists(notebook_path):
        return web.json_response({'error': 'Notebook not found'}, status=400)

    asyncio.create_task(execute_notebook([notebook_path], 'run_digest_generation'))
    return web.json_response({
        'status': 'Digest generation notebook execution started',
        'notebook': notebook_path,
        'message': 'Please check results: https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit?gid=2038294867#gid=2038294867'
    }, status=200)

async def get_progress(request):
    """
    ---
    description: Get the progress of a specific step
    tags:
    - Progress
    parameters:
    - name: token
      in: query
      required: true
      schema:
        type: string
    - name: step
      in: path
      required: true
      schema:
        type: string
    responses:
      '200':
        description: Progress status
      '400':
        description: Invalid step
      '403':
        description: Invalid token
    """
    token = request.query.get('token')
    if token != 'your_secret_token':
        return web.json_response({'error': 'Invalid token'}, status=403)

    step = request.match_info.get('step')
    if step not in progress:
        return web.json_response({'error': 'Invalid step'}, status=400)

    return web.json_response({
        'step': step,
        'progress': progress[step]
    }, status=200)

app = web.Application()

# Swagger документация
swagger = SwaggerDocs(
    app,
    swagger_ui_settings=SwaggerUiSettings(path='/docs'),
    info=SwaggerInfo(
        title="Jupyter Notebook Runner API",
        version="1.0",
        description="An API to run Jupyter notebooks"
    )
)

# Добавление маршрутов
swagger.add_routes([
    web.get('/run_scrape', run_scrape),
    web.get('/run_summarization', run_summarization),
    web.get('/run_grade', run_grade),
    web.get('/run_digest_generation', run_digest_generation),
    web.get('/progress/{step}', get_progress)
])

if __name__ == '__main__':
    # В Jupyter Notebook запускаем приложение через loop.run_until_complete
    loop = asyncio.get_event_loop()
    loop.run_until_complete(web._run_app(app, host='45.133.178.134', port=35474))


======== Running on http://45.133.178.134:35474 ========
(Press CTRL+C to quit)


In [ ]:
1